<a href="https://colab.research.google.com/github/fberanizo/spelling-correction/blob/master/Correcao_T5_Slang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detecção e correção de erro ortográfico com modelo T5

**Nome: Fabio Beranizo Lopes**<br>
**Nome: Luiz Pita Almeida**

Usaremos o modelo T5 pré-treinado e o dataset Paracrawl Inglês-Português. <br>
Truncamos para strings de tamanho 100 para deixar os testes mais rápidos.

Passos:

1. Pressupõe que as frases to Paracrawl estão corretas.<br>
2. Aplica-se todas as técnicas para corromper amostras:
   - Drop
   - Swap
   - Add
   - Key
   - Join
   - Slang
3. Treina-se o modelo T5 com as frases corrompidas como entrada e frase corrigida (original) como saída.

### **Atenção: na técnica Slang reserva-se alguns slangs só para o teste.**

**Obs: os notebooks contém excertos de códigos dos colegas de turma.**<br>
**Obrigado Diedre, Leard e Gabriela.**


In [1]:
import torch

print(f"Current GPU: {torch.cuda.get_device_name(0)}")

# don't even start if it's not a P100 GPU
# if torch.cuda.get_device_name(0) != "Tesla P100-PCIE-16GB":
#     import os
#     os.kill(os.getpid(), 9)

Current GPU: Tesla V100-SXM2-16GB


In [2]:
#@title Configurações gerais
experiment_name = "slang"  #@param {type:"string"}
model_name = "t5-small"  #@param ["t5-small", "t5-base", "t5-large", "t5-3B", "t5-11B"] {type:"string"}
batch_size = 200  #@param {type:"integer"}
accumulate_grad_batches = 1  #@param {type:"integer"}
sequence_length = 100  #@param {type:"integer"}
learning_rate = 5e-3  #@param {type:"number"}
decode_mode = "topk"  #@param ["greedy", "nucleus", "topk", "beam"] {type:"string"}
k = 10  #@param {type:"integer"}
max_epochs = 1  #@param {type:"integer"}

## Instala dependências

- PyTorch Lightning
- Hugginface Transformers
- ERRANT (ERRor ANnotation Toolkit)

In [3]:
!git clone --quiet https://github.com/fberanizo/Adversarial-Misspellings.git

try:
    import pytorch_lightning
    import transformers
except (ImportError, ModuleNotFoundError) as e:
    pass
    # can't import modules, then install
    # !pip install --quiet pytorch-lightning
    # !pip install --quiet transformers
    # !pip install --quiet errant==2.2.0
    # !pip install --quiet google-colab
    # !pip install pyxDamerauLevenshtein
    # !python -m spacy download en
    # kill kernel (necessary for tqdm)
    # import os
    # os.kill(os.getpid(), 9)

In [4]:
# Importar todos os pacotes de uma só vez para evitar duplicados ao longo do notebook.
import datetime
import errant
import gzip
import json
import numpy as np
import nvidia_smi
import os
import pandas as pd
import joblib
import psutil
import pytorch_lightning as pl
import random
import spacy
import sys
import tarfile
import tempfile
import torch
import torch.nn.functional as F

from argparse import Namespace
from collections import deque
# from google.colab import drive
from itertools import cycle
from tqdm import tqdm_notebook

# from pyxdameraulevenshtein import damerau_levenshtein_distance, \
#     normalized_damerau_levenshtein_distance
# from pyxdameraulevenshtein import damerau_levenshtein_distance_ndarray, \
#     normalized_damerau_levenshtein_distance_ndarray

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import Trainer

from transformers import T5ForConditionalGeneration
from transformers import T5Tokenizer
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from typing import Dict
from typing import List
from typing import Tuple

# Leard decoding solution
import html
import unicodedata

nlp = spacy.load("en")
annotator = errant.load("en", nlp)

import nltk
nltk.download("stopwords")

sys.path.insert(0, "/content/Adversarial-Misspellings")
import attacks

nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)

def hardware_stats():
    """
    Returns a dict containing some hardware related stats
    """
    res = nvidia_smi.nvmlDeviceGetUtilizationRates(handle)
    return {"cpu": f"{str(psutil.cpu_percent())}%",
            "mem": f"{str(psutil.virtual_memory().percent)}%",
            "gpu": f"{str(res.gpu)}%",
            "gpu_mem": f"{str(res.memory)}%"}

[nltk_data] Downloading package stopwords to /home/fabiol/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Define random seeds

Importante: Fix seeds so we can replicate results

In [5]:
import random

seed = 0
random.seed(seed)
torch.random.manual_seed(seed)
torch.cuda.manual_seed(seed)

## Mapeia Google Drive

Iremos salvar os checkpoints (pesos do modelo) no google drive, para que possamos continuar o treino de onde paramos.

In [6]:
# drive.mount("/content/drive")
# base_path = "/content/drive/My Drive/PF-Correcao/paracrawl-t5-slang"
base_path = "/content/paracrawl-t5-slang"
os.environ["BASE_PATH"] = base_path

## Classe Dataset
Gerenciamento dos dados, e um pequeno teste.
No getitem é aplicada a correção de codificação.

In [ ]:
hparams = {"model_name": model_name, "seq_len": sequence_length, "batch_size": batch_size}
class ParaCrawl(Dataset):
    """
    Loads data from preprocessed file and manages them.
    """
    VALID_MODES = ["train", "validation", "test"]
    TOKENIZER = T5Tokenizer.from_pretrained(hparams["model_name"],
                                            cache_dir=base_path)
    def __init__(self, mode: str, seq_len: int):
        """
        mode: One of train, validation or test 
        seq_len: limit to returned encoded tokens
        """
        super().__init__()
        assert mode in ParaCrawl.VALID_MODES

        self.mode = mode
        self.seq_len = seq_len

        file_name = os.path.join(base_path, f"{mode}.pkl")
        if not os.path.isfile(file_name):
            print("Pre-processed files not found, preparing data.")
            self.prepare_data()
        
        with open(file_name, "rb") as preprocessed_file:
            self.data = joblib.load(preprocessed_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i: int):
        """
        Unpacks line from data.

        returns: input (corrputed), target (corrected)
        """
        input, target = self.data[i]

        input_normalized = unicodedata.normalize("NFD", input).\
            encode("latin-1", "xmlcharrefreplace").\
            decode("latin-1")

        input_encoded = ParaCrawl.TOKENIZER.encode_plus(
            f"{input_normalized} {ParaCrawl.TOKENIZER.eos_token}",
            max_length=self.seq_len,
            pad_to_max_length=True,
            return_token_type_ids=False,
            return_tensors="pt")

        input_ids = input_encoded["input_ids"].squeeze()
        input_mask = input_encoded["attention_mask"].squeeze()

        target_normalized = unicodedata.normalize("NFD", target).\
            encode("latin-1", "xmlcharrefreplace").\
            decode("latin-1")

        target_encoded = ParaCrawl.TOKENIZER.encode_plus(
            f"{target_normalized} {ParaCrawl.TOKENIZER.eos_token}",
            max_length=self.seq_len,
            pad_to_max_length=True,
            return_token_type_ids=False,
            return_tensors="pt")

        target_ids = target_encoded["input_ids"].squeeze()
        target_mask = target_encoded["attention_mask"].squeeze()

        return input, target, input_ids, input_mask, target_ids, target_mask

    def get_dataloader(self, batch_size: int, shuffle: bool):
        return DataLoader(self, batch_size=batch_size, shuffle=shuffle, 
                          num_workers=4)

    @staticmethod
    def load_text_pairs(path):
        """
        Load pairs from original files, selects pt, then corrupts the samples.
        """
        text_pairs = []
        for line in tqdm_notebook(gzip.open(path, mode="rt")):
            text_pt = line.strip().split("\t")[1]
            text_pt = text_pt[:hparams["seq_len"]].rsplit(" ", 1)[0]
            try:
                attack_list = deque(attacks.all_one_attack(text_pt, include_ends=True))
                text_corrupt = list(map(lambda a: a[1], random.sample(attack_list, k=10)))
                text_pairs.extend(list(zip(text_corrupt, cycle([text_pt]))))
            except:
                pass

        return text_pairs

    @staticmethod
    def prepare_data(train_size=9997000, val_size=3000):
        """
        Performs everything needed to get the data ready.
        Addition of Eos token and encoding is performed in runtime.
        """
        if not os.path.isfile("paracrawl_enpt_train.tsv.gz"):    
            !wget -nc https://storage.googleapis.com/neuralresearcher_data/unicamp/ia376e_2020s1/paracrawl_enpt_train.tsv.gz -P "$BASE_PATH"
            !wget -nc https://storage.googleapis.com/neuralresearcher_data/unicamp/ia376e_2020s1/paracrawl_enpt_test.tsv.gz -P "$BASE_PATH"

        data = {}
        test_data = ParaCrawl.load_text_pairs(os.path.join(base_path, "paracrawl_enpt_test.tsv.gz"))
        train_val_data = ParaCrawl.load_text_pairs(os.path.join(base_path, "paracrawl_enpt_train.tsv.gz"))

        random.shuffle(train_val_data)

        train_data = train_val_data[:train_size]
        val_data = train_val_data[train_size:train_size + val_size]

        for mode, data in zip(ParaCrawl.VALID_MODES, [train_data, val_data, test_data]):
            file_name = os.path.join(base_path, f"{mode}.pkl")
            with open(file_name, "wb") as pkl_file:
                joblib.dump(data, pkl_file)
            print(f"Pre-processed data saved as {file_name}.")


datasets = {m: ParaCrawl(mode=m, seq_len=hparams["seq_len"]) for m in ParaCrawl.VALID_MODES}

# Testing datasets
for mode, dataset in datasets.items():
    print(f"\n{mode} dataset length: {len(dataset)}\n")
    print("Random sample")
    input, target, input_ids, input_mask, target_ids, target_mask = random.choice(dataset)
    print("input\n", input, end="\n\n")
    print("target\n", target, end="\n\n")


Pre-processed files not found, preparing data.
--2020-06-24 15:51:48--  https://storage.googleapis.com/neuralresearcher_data/unicamp/ia376e_2020s1/paracrawl_enpt_train.tsv.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.128, 2607:f8b0:4001:c0c::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106548256 (102M) [text/tab-separated-values]
Saving to: ‘/content/paracrawl-t5-slang/paracrawl_enpt_train.tsv.gz’

paracrawl_enpt_trai 100%[===================>] 101.61M  40.7MB/s    in 2.5s    

2020-06-24 15:51:51 (40.7 MB/s) - ‘/content/paracrawl-t5-slang/paracrawl_enpt_train.tsv.gz’ saved [106548256/106548256]

--2020-06-24 15:51:51--  https://storage.googleapis.com/neuralresearcher_data/unicamp/ia376e_2020s1/paracrawl_enpt_test.tsv.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.119.128, 2607:f8b0:4001:c0d::80
Connecting to storage.goog

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:79: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Pre-processed data saved as /content/paracrawl-t5-slang/train.pkl.
Pre-processed data saved as /content/paracrawl-t5-slang/validation.pkl.
Pre-processed data saved as /content/paracrawl-t5-slang/test.pkl.


## Dataloaders

Verificação se dataloaders estão funcionando corretamente.

In [ ]:
shuffle = {"train": True, "validation": False, "test": False}
debug_dataloaders = {mode: datasets[mode].get_dataloader(batch_size=hparams["batch_size"], 
                                                         shuffle=shuffle[mode])
                     for mode in ParaCrawl.VALID_MODES}

# Testing dataloaders
for mode, dataloader in debug_dataloaders.items():
    print("{} number of batches: {}".format(mode, len(dataloader)))
    batch = next(iter(dataloader))

train number of batches: 99970
validation number of batches: 27
test number of batches: 2000


## Lightning Module

Aqui a classe principal do PyTorch Lightning é definida.


In [ ]:
class T5Corrector(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()

        self.hparams = hparams
        self.t5 = T5ForConditionalGeneration.from_pretrained(hparams.model_name,
                                                             cache_dir=hparams.base_path)
        self.tokenizer = ParaCrawl.TOKENIZER
        self.start_token = ParaCrawl.TOKENIZER.convert_tokens_to_ids("<extra_id_0>")
        self.end_token = ParaCrawl.TOKENIZER.convert_tokens_to_ids("<extra_id_1>")

    def forward(self, x):
        inputs, targets, input_ids, input_mask, target_ids, target_mask = x

        if self.training:
            outputs = self.t5(input_ids=input_ids, 
                              attention_mask=input_mask,
                              lm_labels=target_ids)
            loss, predicted_scores = outputs[:2]
            return loss, predicted_scores, inputs, targets
        else:
            outputs = self.t5.generate(input_ids=input_ids, 
                                       attention_mask=input_mask,
                                       lm_labels=target_ids,
                                       max_length=self.hparams.seq_len)
            return outputs, inputs, targets

    def training_step(self, batch, targets):
        loss, predicted_scores, inputs, targets = self(batch)

        return {"loss": loss, "log": {"loss": loss}, "progress_bar": hardware_stats()}

    def validation_step(self, batch, batch_idx):
        outputs, inputs, targets = self(batch)

        predicts = [
            html.unescape(
                self.tokenizer.decode(output_ids,
                                      skip_special_tokens=False,
                                      clean_up_tokenization_spaces=False)
            )
            for output_ids in outputs]

        with open("orig.txt", "w") as f:
            for input in inputs:
                input = input.replace("\n", "")
                f.write(f"{input}\n")

        with open("ref.txt", "w") as f:
            for target in targets:
                target = target.replace("\n", "")
                f.write(f"{target}\n")

        with open("hyp.txt", "w") as f:
            for pred in predicts:
                pred = pred.replace("\n", "")
                f.write(f"{pred}\n")

        !errant_parallel -orig orig.txt -cor ref.txt -out ref.m2 > /dev/null
        !errant_parallel -orig orig.txt -cor hyp.txt -out hyp.m2 > /dev/null
        x = !errant_compare -hyp hyp.m2 -ref ref.m2
        df = pd.DataFrame(data=x[2:4])[0].str.split("\t", expand=True)
        new_header = df.iloc[0] #grab the first row for the header
        df = df[1:].apply(pd.to_numeric) #take the data less the header row
        df.columns = new_header

        true_positive = df["TP"][1]
        false_positive = df["FP"][1]
        false_negative = df["FN"][1]
        precision = df["Prec"][1]
        recall = df["Rec"][1]
        f_score = df["F0.5"][1]

        progress_bar = hardware_stats()
        progress_bar.update({"precision": precision, "recall": recall, "f_score": f_score})

        return {"true_positive": true_positive, "false_positive": false_positive, "false_negative": false_negative,
                "precision": precision, "recall": recall, "f_score": f_score,
                "predicts": predicts, "inputs": inputs, "targets": targets, "progress_bar": progress_bar}

    def test_step(self, batch, batch_idx):
        outputs, inputs, targets = self(batch)

        predicts = [
            html.unescape(
                self.tokenizer.decode(output_ids,
                                      skip_special_tokens=False,
                                      clean_up_tokenization_spaces=False)
            )
            for output_ids in outputs]

        with open("orig.txt", "w") as f:
            for input in inputs:
                input = input.replace("\n", "")
                f.write(f"{input}\n")

        with open("ref.txt", "w") as f:
            for target in targets:
                target = target.replace("\n", "")
                f.write(f"{target}\n")

        with open("hyp.txt", "w") as f:
            for pred in predicts:
                pred = pred.replace("\n", "")
                f.write(f"{pred}\n")

        !errant_parallel -orig orig.txt -cor ref.txt -out ref.m2 > /dev/null
        !errant_parallel -orig orig.txt -cor hyp.txt -out hyp.m2 > /dev/null
        x = !errant_compare -hyp hyp.m2 -ref ref.m2
        df = pd.DataFrame(data=x[2:4])[0].str.split("\t", expand=True)
        new_header = df.iloc[0] #grab the first row for the header
        df = df[1:].apply(pd.to_numeric) #take the data less the header row
        df.columns = new_header

        true_positive = df["TP"][1]
        false_positive = df["FP"][1]
        false_negative = df["FN"][1]
        precision = df["Prec"][1]
        recall = df["Rec"][1]
        f_score = df["F0.5"][1]

        progress_bar = hardware_stats()
        progress_bar.update({"precision": precision, "recall": recall, "f_score": f_score})

        return {"true_positive": true_positive, "false_positive": false_positive, "false_negative": false_negative,
                "precision": precision, "recall": recall, "f_score": f_score,
                "predicts": predicts, "inputs": inputs, "targets": targets, "progress_bar": progress_bar}

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x["loss"] for x in outputs]).mean()

        return {"log": {"train_loss": avg_loss}} 

    def validation_epoch_end(self, outputs):
        avg_precision = sum([x["precision"] for x in outputs]) / len(outputs)
        avg_recall = sum([x["recall"] for x in outputs]) / len(outputs)
        avg_f_score = sum([x["f_score"] for x in outputs]) / len(outputs)

        tensorboard_logs = {"val_loss": avg_f_score,
                            "avg_precision": avg_precision,
                            "avg_recall": avg_recall,
                            "avg_f_score": avg_f_score}

        origs = sum([list(x["inputs"]) for x in outputs], [])
        trues = sum([list(x["targets"]) for x in outputs], [])
        preds = sum([list(x["predicts"]) for x in outputs], [])

        n = random.choice(range(len(trues)))
        print(f"\Input: {origs[n]}\nTarget: {trues[n]}\nPrediction: {preds[n]}\n")

        return {"val_loss": avg_f_score, "avg_precision": avg_precision, "avg_recall": avg_recall, "avg_f_score": avg_f_score,
                "log": tensorboard_logs, "progress_bar": tensorboard_logs}

    def test_epoch_end(self, outputs):
        avg_precision = sum([x["precision"] for x in outputs]) / len(outputs)
        avg_recall = sum([x["recall"] for x in outputs]) / len(outputs)
        avg_f_score = sum([x["f_score"] for x in outputs]) / len(outputs)

        tensorboard_logs = {"avg_precision": avg_precision,
                            "avg_recall": avg_recall,
                            "avg_f_score": avg_f_score}

        origs = sum([list(x["inputs"]) for x in outputs], [])
        trues = sum([list(x["targets"]) for x in outputs], [])
        preds = sum([list(x["predicts"]) for x in outputs], [])

        n = random.choice(range(len(trues)))
        print(f"\Input: {origs[n]}\nTarget: {trues[n]}\nPrediction: {preds[n]}\n")
        
        return {"avg_precision": avg_precision, "avg_recall": avg_recall, "avg_f_score": avg_f_score,
                "log": tensorboard_logs, "progress_bar": tensorboard_logs}

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=self.hparams.lr)    

    def train_dataloader(self):
        if self.hparams.overfit_pct > 0:
            logging.info("Disabling train shuffle due to overfit_pct.")
            shuffle = False
        else:
            shuffle = True
        dataset = ParaCrawl("train", seq_len=self.hparams.seq_len)
        return dataset.get_dataloader(batch_size=self.hparams.batch_size, shuffle=shuffle)

    def val_dataloader(self):
        dataset = ParaCrawl("validation", seq_len=self.hparams.seq_len)
        return dataset.get_dataloader(batch_size=self.hparams.batch_size, shuffle=False)

    def test_dataloader(self):
        dataset = ParaCraw("test", seq_len=self.hparams.seq_len)
        return dataset.get_dataloader(batch_size=self.hparams.batch_size, shuffle=False)

## Preparação e Treino
Define-se hiperparâmetros para um treinamento de melhora, agora com decodificação corrigida.

In [ ]:
hparams = {"name": experiment_name, "base_path": base_path,
           "model_name": model_name, "seq_len": sequence_length,
           "decode_mode": decode_mode, "k": k,
           "lr": learning_rate, "batch_size": batch_size, "batch_accum": accumulate_grad_batches,
           "max_epochs": max_epochs,
           "overfit_pct": 0, "debug": 0,
           "decode_mode": decode_mode}

for key, parameter in hparams.items():
    print("{}: {}".format(key, parameter))

name: slang
base_path: /content/paracrawl-t5-slang
model_name: t5-small
seq_len: 100
decode_mode: topk
k: 10
lr: 0.005
batch_size: 100
batch_accum: 1
max_epochs: 10
overfit_pct: 0
debug: 0


In [ ]:
# Instantiate model
model = T5Corrector(Namespace(**hparams))

# Folder/path management, for logs and checkpoints
tensorboard_path = os.path.join(base_path, "logs")
experiment_name = hparams["name"]
model_folder = os.path.join(tensorboard_path, experiment_name)
os.makedirs(model_folder, exist_ok=True)
ckpt_path = os.path.join(model_folder, "-{epoch}")

# Callback initialization
checkpoint_callback = ModelCheckpoint(prefix=experiment_name, 
                                      filepath=ckpt_path, 
                                      mode="max",
                                      verbose=True)
logger = TensorBoardLogger(tensorboard_path, experiment_name, version=0)

# PL Trainer initialization
trainer = Trainer(gpus=1,
                  checkpoint_callback=checkpoint_callback, 
                  early_stop_callback=False,
                  logger=logger,
                  accumulate_grad_batches=hparams["batch_accum"],
                  max_epochs=hparams["max_epochs"], 
                  fast_dev_run=bool(hparams["debug"]), 
                  overfit_pct=hparams["overfit_pct"],
                  progress_bar_refresh_rate=10,
                  val_check_interval=0.1, # check validation set 4 times during a training epoch
                  profiler=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


## Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "/content/paracrawl-t5-slang"
# %tensorboard --logdir "/content/drive/My Drive/PF-Correcao/paracrawl-t5-slang"

## Fine-tuning

In [ ]:
trainer.fit(model)


  | Name | Type                       | Params
----------------------------------------------------
0 | t5   | T5ForConditionalGeneration | 60 M  


\Input: Há 1 mês 22:06 xHzamster milf, italiano, meias finas,
Target: Há 1 mês 22:06 xHamster milf, italiano, meias finas,
Prediction: Há 1 mês 22:06 xHzamster milf, italiano, meias finas, meias finas, meias finas, meias finas, meias finas, meias finas, meias finas, meias finas, meias finas,



\Input: Voos para Vazncouver - Ofertas de voos baratos para Vancouver, Canadá com
Target: Voos para Vancouver - Ofertas de voos baratos para Vancouver, Canadá com
Prediction: Voos para Vancouver - Ofertas de voos baratos para Vancouver, Canadá com



/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: The metric you returned None must be a `torch.Tensor` instance, checkpoint not saved HINT: what is the value of val_loss in validation_epoch_end()?
  warnings.warn(*args, **kwargs)
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: Can save best model only with val_loss available, skipping.
  warnings.warn(*args, **kwargs)


\Input: Oremos a Nossa Senhora: eAve
Target: Oremos a Nossa Senhora: Ave
Prediction: Oremos a Nossa Senhora: Ave



\Input: Muateriais de construção em Peru -
Target: Materiais de construção em Peru -
Prediction: Materiais de construção em Peru -



\Input: AI antagonlistas & inibidores AA análogos &
Target: AI antagonistas & inibidores AA análogos &
Prediction: AI antagonistas & inibidores AA análogos &



\Input: Salas Restaurante Grill , localizado no coração de Sevilha , tornou-se um epaço cardioprotegido .
Target: Salas Restaurante Grill , localizado no coração de Sevilha , tornou-se um espaço cardioprotegido .
Prediction: Salas Restaurante Grill , localizado no coração de Sevilha , tornou-se um espaço cardioprotegido .



\Input: As respectivas influências estdão de acordo com o seguinte quadro:
Target: As respectivas influências estão de acordo com o seguinte quadro:
Prediction: As respectivas influências estão de acordo com o seguinte quadro:



\Input: nCerâmica em Mendoza,
Target: Cerâmica em Mendoza,
Prediction: Cerâmica em Mendoza,



## Testes

In [ ]:
trainer.test(model)